# Reflection and Blogpost Writing

## Setup

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
#print(OPENAI_API_KEY)
#llm_config = {"model": "gpt-3.5-turbo","api_key":OPENAI_API_KEY}
llm_config = {"model": "gemini-1.5-flash","api_key":"PutYouRKey","api_type":"google"}
#llm_config = {"model": "llama3","api_type":"ollama"}


## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
      Pollution . Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [7]:
reply = writer.generate_reply(messages=[{"content": task ,"role": "user"}])

In [8]:
print(reply)

{'content': "## Hope AI: A Glimpse into a Brighter Future?\n\nHope AI isn't just another buzzword; it's a burgeoning field exploring AI's potential for social good.  Imagine AI tackling climate change, improving healthcare access, or even ending world hunger.  While challenges remain, the ethical development and deployment of AI offer a powerful tool to address humanity's greatest challenges, sparking hope for a more equitable and sustainable future.  The journey is complex, but the potential rewards are immense.\n", 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None}


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [9]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [10]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

## Hope AI: A Glimpse into a Brighter Future?

Hope AI isn't just another buzzword; it represents a powerful shift in artificial intelligence.  Instead of focusing solely on efficiency, Hope AI prioritizes ethical considerations and human well-being.  This means AI systems designed not just to solve problems, but to do so in a way that benefits humanity, fostering inclusivity and minimizing harm.  While still nascent, Hope AI offers a compelling vision: a future where technology serves as a force for good, empowering individuals and communities worldwide.


--------------------------------------------------------------------------------
Critic (to Writer):

This is a good start to a blog post about Hope AI, hitting the key points concisel

## Nested chat

In [11]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO_Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [12]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal_Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [13]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics_Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [14]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta_Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [15]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [16]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [17]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

## Hope AI: A Glimpse into a Brighter Future?

Hope AI isn't just another buzzword; it represents a powerful shift in artificial intelligence.  Instead of focusing solely on efficiency, Hope AI prioritizes ethical considerations and human well-being.  This means AI systems designed not just to solve problems, but to do so in a way that benefits humanity, fostering inclusivity and minimizing harm.  While still nascent, Hope AI offers a compelling vision: a future where technology serves as a force for good, empowering individuals and communities worldwide.


--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat..

## Get the summary

In [18]:
print(res.summary)

## Hope AI: Building a Better Future with Ethical AI

Hope AI isn't just a concept; it's a movement building ethical and responsible AI solutions.  Instead of prioritizing profit alone, Hope AI focuses on projects that demonstrably improve human lives. For example, [insert specific example of Hope AI project 1, with quantifiable results], showcasing the positive impact of human-centered design. Another initiative, [insert specific example of Hope AI project 2, with quantifiable results], highlights how AI can address pressing social challenges.  While still in its early stages of development – with key milestones including [mention specific milestones], and future goals focusing on [mention future goals] – Hope AI is actively shaping a future where AI empowers individuals and communities.  Learn more and join the movement at [link to website/initiative].

